In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyabf
from scipy.signal import find_peaks
import plotly.express as px
import plotly.graph_objects as go
import fxn as mabf

In [ ]:
#### Change to the directory with input file to work ####
input_data_dir = "C:/work/Broad/input/18/"
output_data_dir = "C:/work/Broad/output/"
filenames = os.listdir(input_data_dir)
#### change index to pick file ####
filename =filenames[21]
filepath = input_data_dir + '/' + filename

abf = pyabf.ABF(filepath)
base = os.path.basename(filepath)
name = os.path.splitext(base)[0]

In [ ]:
# set sweep
sweep =0
abf.setSweep(sweep)
time = abf.sweepX
time_resolution =  time[1]  
freq = round(1/time_resolution) 
voltage = abf.sweepY
current = abf.sweepC
voltage_derivative = mabf.get_derivative(voltage, x_unit_distance = time_resolution)
print(f'sweep count: {abf.sweepCount}, freq : {freq} Hz, sample time: {time_resolution*1000} ms, period: {time[-1]:.2f} sec')
# stimulus start end 
stim_s = mabf.get_index_sharp_change(current, threshold=-10, window=1)
stim_e = mabf.get_index_sharp_change(current, threshold=10, window=1)
print(f'Stimulus starts at {stim_s} end at {stim_e}')

# RMP 
rmp_s, rmp_e, rmp = mabf.get_resting_membrane_potential(voltage)
print(f'RMP: {rmp:.2f} mV, averaging from {rmp_s} - {rmp_e}')
fhv = voltage[0]

# hyperpolarization
hpp_s, hpp_e, hpp_ind, hpp = mabf.get_hyperpolarized_potential(voltage, current)
hp_amp = hpp - rmp
print(f'RMP={rmp:.2f} mV, HPP={hpp:.2f} mV, HP_AMP={hp_amp:.2f} mV, current amp={current.min()} mA, first_holding_waveform  (mV): {fhv}')
print(f'Hyperpolarization start at {hpp_s}, end at {hpp_e}')

hpp_points = [rmp_s, rmp_e, hpp_s, hpp_e, hpp_ind]
hpp_labels = ['rmp_start', 'rmp_end', 'hpp_start', 'hpp_end', 'hpp_bottom']
hpp_df = pd.DataFrame({'label': hpp_labels, 'point_index': hpp_points})

sweep_info = mabf.Sweep(dt=time_resolution, period=round(time[-1],2), rmp=rmp, 
            hpp=hpp, hp_amp=hp_amp, hpp_s=hpp_s, hpp_e=hpp_e)

In [ ]:
# plot the whole sweep, WARNING it may take some to plot all data  
f2 = mabf.plot_sweep(time, current, voltage)
f2.show()

In [ ]:
# plot the hpp
fig =  mabf.plot_hpp(time, current, voltage, hpp_df)
fig.show()

In [ ]:
# find peaks 
peak_indices, _ = find_peaks(voltage, height=-20, threshold=-40)
peak_indices = np.array(peak_indices)
if len(peak_indices) > 0:
    print(f'peak num: {len(peak_indices)},  peak start: {peak_indices[0]}, peak end: {peak_indices[-1]}, hyperpolarization end: {hpp_e}')
    print(f'peak starts after hpp ends: {hpp_e < peak_indices[0]}')
    print(f'peak voltages: {voltage[peak_indices]}')
    # get bursts info 
    df_peak_info = mabf.get_bursts(peak_indices, time_resolution)
else:
    print('No peak detected')
    # get bursts info 
    df_peak_info = None

In [ ]:
if df_peak_info is not None:
    df_burst_info = df_peak_info.loc[~df_peak_info.burst_index.isna()]
    #### change the index to pick different burst ####
    current_burst_index = 2
    current_burst  = df_burst_info[df_burst_info['burst_index'] == current_burst_index]
    next_burst_index = current_burst_index + 1 
    if next_burst_index < max(df_burst_info['burst_index']):
        tmp_end = df_burst_info[df_burst_info['burst_index'] == next_burst_index].peak_index.iloc[0]
    else:
        tmp_end =  df_burst_info[df_burst_info['burst_index'] == current_burst_index].peak_index.iloc[-1]+1000
        tmp_end = min(tmp_end, len(voltage)-1)
    burst_detail = mabf.get_burst_detail(current_burst, time, voltage, voltage_derivative, sweep_info, tmp_end) 
    burst_fig =  mabf.plot_burst(time, voltage, sweep_info, current_burst, burst_detail)
    burst_fig.show()